In [ ]:
%tensorflow_version 1.x
import tensorflow as tf 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

TensorFlow 1.x selected.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


1. 检查每个特征的缺失值的数量
2. 将X和y分开
3. 删去缺失值超过一半的特征
4. 填入缺失值
5. 将数值数据标准化
6. 将类别数据进行onehot编码

载入训练集和测试集，分别存入train_data和test_data。
train_data的最后一列为“房屋价格”。
test_data比train_data少一列，即test_data没有“房屋价格”这一列。

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
print("The shape of train_data: {}".format(train_data.shape))
print("The shape of test_data:  {}".format(test_data.shape))
train_data.head()

The shape of train_data: (1460, 81)
The shape of test_data:  (1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


将train_data的最后一列（“房屋价格”）删去，并把train_data和test_data进行合并，组成一个大表格，对这个大数据集进行数据预处理。

In [ ]:
data = pd.concat([train_data.drop('SalePrice',axis=1), test_data], axis=0)
print(data.shape)
data.head()

(2919, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [ ]:
index_list = train_data.columns
print(index_list)

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

编写函数


*   analysis_feature函数分析传入的特征的缺失值数量，unique_value和数据类型
*   unique_val_count函数计算传入的特征的出现次数最多的值和它出现的次数
*   sep_missing_value_feature函数将有缺失值的特征名存入列表contain_missing_val，将无缺失值的特征名存入no_missing_val。
*   sep_numerical_feature函数将数值型特征的特征名存入列表numerical_feature，将其他类型特征的特征名存入列表string_feature。
*   onehot函数对传入的数据进行one-hot编码
*   impute_numerical_data函数处理数值型特征的缺失值，用每个特征的平均值来填补缺失值。
*   combine2df函数将两个行数相同的表格按列拼接到一起。




In [ ]:
def analysis_feature(input_data, feature_name):
  feature_series = input_data[feature_name]
  missing_val_num = np.sum(feature_series.isnull())
  unique_val = feature_series.unique()
  val_type = feature_series.dtype

  print("Missing value num: {}".format(missing_val_num))
  print("Unique value:      {}".format(unique_val))
  print("Value type:        {}".format(val_type))

  return missing_val_num, unique_val, val_type

def unique_val_count(input_data, feature_name, unique_val):
  most_frequent_times = -1
  most_frequent = None
  result = dict()
  for val in unique_val:
    times = np.sum(input_data[feature_name]==val)
    result[val] = times
    if times > most_frequent_times:
      most_frequent_times = times
      most_frequent = val
  
  print("The most frequent val is: {}, it appeared {} times.".format(most_frequent, most_frequent_times))

  return result

In [ ]:
def sep_missing_value_feature(input_data):
  feature_list = input_data.columns
  no_missing_val = []
  contain_missing_val = []
  for feature in feature_list:
    if np.sum(input_data[feature].isnull()) == 0:
      no_missing_val.append(feature)
    else:
      contain_missing_val.append(feature)

  return no_missing_val, contain_missing_val

def sep_numerical_feature(input_data):
  feature_list = input_data.columns
  numerical_feature = []
  string_feature = []
  for feature in feature_list:
    if input_data[feature].dtype != 'object':
      numerical_feature.append(feature)
    else:
      string_feature.append(feature)
  return numerical_feature, string_feature

def onehot(input_data):
  result = pd.get_dummies(input_data)
  return result

def impute_numerical_data(input_data):
  feature_list = input_data.columns
  result = pd.DataFrame()

  for feature in feature_list:
    if input_data[feature].dtype == 'object':
      print("Feature " + str(feature) + " is not numerical!")
      return
    
    mean_val = input_data[feature].mean()
    result[feature] = input_data[feature].replace(np.nan, mean_val)
  
  for feature in result.columns:
    if np.sum(result[feature].isnull()) != 0:
      print("Feature" + str(feature) + " still contain null values!")
      return
  
  print("Impute with mean value complete!")
  return result

def combine2df(df1, df2):
  feature_list1 = df1.columns
  feature_list2 = df2.columns
  result = pd.DataFrame()
  for feature in feature_list1:
    result[feature] = df1[feature]
  for feature in feature_list2:
    result[feature] = df2[feature]
  
  return result

* 首先将有缺失值的特征和无缺失值的特征分开。
* 然后将有缺失值的特征分为数值型特征和其他类型特征。

In [ ]:
no_missing_val_feature, missing_val_feature = sep_missing_value_feature(data)
missing_num_val_feature, missing_str_val_feature = sep_numerical_feature(data[missing_val_feature])

print("Missing val feature: {}".format(missing_val_feature))
print("Missing num val feature: {}".format(missing_num_val_feature))
print("Missing str val feature: {}".format(missing_str_val_feature))

Missing val feature: ['MSZoning', 'LotFrontage', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']
Missing num val feature: ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']
Missing str val feature: ['MSZoning', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeat

用每个特征的平均值填补缺失值

In [ ]:
# Impute numerical missing feature
print("Featrues being processed: {}".format(missing_num_val_feature))
imputed_num_feature = impute_numerical_data(data[missing_num_val_feature])
imputed_num_feature.head()

Featrues being processed: ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea']
Impute with mean value complete!


,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
0,65.0,196.0,706.0,0.0,150.0,856.0,1.0,0.0,2003.0,2.0,548.0
1,80.0,0.0,978.0,0.0,284.0,1262.0,0.0,1.0,1976.0,2.0,460.0
2,68.0,162.0,486.0,0.0,434.0,920.0,1.0,0.0,2001.0,2.0,608.0
3,60.0,0.0,216.0,0.0,540.0,756.0,1.0,0.0,1998.0,3.0,642.0
4,84.0,350.0,655.0,0.0,490.0,1145.0,1.0,0.0,2000.0,3.0,836.0


将没有缺失值的数值型特征和已填补缺失值的数值型特征拼在一起。至此，所有数值型特征已经预处理完毕。

In [ ]:
# Combine numerical feature
no_missing_num_val_feature, _ = sep_numerical_feature(data[no_missing_val_feature])
print("Shape of no_missing_num_val_feature: {}".format(data[no_missing_num_val_feature].shape))
print("Shape of imputed_num_feature: {}".format(imputed_num_feature.shape))
numerical_features = combine2df(data[no_missing_num_val_feature], imputed_num_feature)
print(numerical_features.shape)
numerical_features.head()

Shape of no_missing_num_val_feature: (2919, 26)
Shape of imputed_num_feature: (2919, 11)
(2919, 37)


,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
0,1,60,8450,7,5,2003,2003,856,854,0,1710,2,1,3,1,8,0,0,61,0,0,0,0,0,2,2008,65.0,196.0,706.0,0.0,150.0,856.0,1.0,0.0,2003.0,2.0,548.0
1,2,20,9600,6,8,1976,1976,1262,0,0,1262,2,0,3,1,6,1,298,0,0,0,0,0,0,5,2007,80.0,0.0,978.0,0.0,284.0,1262.0,0.0,1.0,1976.0,2.0,460.0
2,3,60,11250,7,5,2001,2002,920,866,0,1786,2,1,3,1,6,1,0,42,0,0,0,0,0,9,2008,68.0,162.0,486.0,0.0,434.0,920.0,1.0,0.0,2001.0,2.0,608.0
3,4,70,9550,7,5,1915,1970,961,756,0,1717,1,0,3,1,7,1,0,35,272,0,0,0,0,2,2006,60.0,0.0,216.0,0.0,540.0,756.0,1.0,0.0,1998.0,3.0,642.0
4,5,60,14260,8,5,2000,2000,1145,1053,0,2198,2,1,4,1,9,1,192,84,0,0,0,0,0,12,2008,84.0,350.0,655.0,0.0,490.0,1145.0,1.0,0.0,2000.0,3.0,836.0


接下来处理有缺失值的其他特征。对于一下每一个特征，首先分析它的缺失值数量，特征中出现过的值，以及特征中出现次数最多的值。根据每个特征描述的内容，分别用出现次数最多的值和None填补缺失值。

In [ ]:
# Impute str missing feature
print("Featrues being processed: {}".format(missing_str_val_feature))

Featrues being processed: ['MSZoning', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']


In [ ]:
#      ['MSZoning', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual', 
#       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 
#       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']

#####'MSZoning'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'MSZoning')

val_count = unique_val_count(data, 'MSZoning', unique_val)
print(val_count)

MSZoning = data['MSZoning'].replace(np.nan, 'RL')
print(np.sum(MSZoning.isnull()))

#####'Alley'#####
missing_val_num, unique_val, val_type = analysis_feature(train_data, 'Alley')
#drop

Missing value num: 4
Unique value:      ['RL' 'RM' 'C (all)' 'FV' 'RH' nan]
Value type:        object
The most frequent val is: RL, it appeared 2265 times.
{'RL': 2265, 'RM': 460, 'C (all)': 25, 'FV': 139, 'RH': 26, nan: 0}
0
Missing value num: 1369
Unique value:      [nan 'Grvl' 'Pave']
Value type:        object


In [ ]:
#####'Utilities'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'Utilities')

val_count = unique_val_count(data, 'Utilities', unique_val)
print(val_count)

Utilities = data['Utilities'].replace(np.nan, 'AllPub')
print(np.sum(Utilities.isnull()))

Missing value num: 2
Unique value:      ['AllPub' 'NoSeWa' nan]
Value type:        object
The most frequent val is: AllPub, it appeared 2916 times.
{'AllPub': 2916, 'NoSeWa': 1, nan: 0}
0


In [ ]:
#####'Exterior1st'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'Exterior1st')

val_count = unique_val_count(data, 'Exterior1st', unique_val)
print(val_count)

Exterior1st = data['Exterior1st'].replace(np.nan, 'VinylSd')
print(np.sum(Exterior1st.isnull()))

Missing value num: 1
Unique value:      ['VinylSd' 'MetalSd' 'Wd Sdng' 'HdBoard' 'BrkFace' 'WdShing' 'CemntBd'
 'Plywood' 'AsbShng' 'Stucco' 'BrkComm' 'AsphShn' 'Stone' 'ImStucc'
 'CBlock' nan]
Value type:        object
The most frequent val is: VinylSd, it appeared 1025 times.
{'VinylSd': 1025, 'MetalSd': 450, 'Wd Sdng': 411, 'HdBoard': 442, 'BrkFace': 87, 'WdShing': 56, 'CemntBd': 126, 'Plywood': 221, 'AsbShng': 44, 'Stucco': 43, 'BrkComm': 6, 'AsphShn': 2, 'Stone': 2, 'ImStucc': 1, 'CBlock': 2, nan: 0}
0


In [ ]:
#####'Exterior2nd'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'Exterior2nd')

val_count = unique_val_count(data, 'Exterior2nd', unique_val)
print(val_count)

Exterior2nd = data['Exterior2nd'].replace(np.nan, 'VinylSd')
print(np.sum(Exterior2nd.isnull()))

Missing value num: 1
Unique value:      ['VinylSd' 'MetalSd' 'Wd Shng' 'HdBoard' 'Plywood' 'Wd Sdng' 'CmentBd'
 'BrkFace' 'Stucco' 'AsbShng' 'Brk Cmn' 'ImStucc' 'AsphShn' 'Stone'
 'Other' 'CBlock' nan]
Value type:        object
The most frequent val is: VinylSd, it appeared 1014 times.
{'VinylSd': 1014, 'MetalSd': 447, 'Wd Shng': 81, 'HdBoard': 406, 'Plywood': 270, 'Wd Sdng': 391, 'CmentBd': 126, 'BrkFace': 47, 'Stucco': 47, 'AsbShng': 38, 'Brk Cmn': 22, 'ImStucc': 15, 'AsphShn': 4, 'Stone': 6, 'Other': 1, 'CBlock': 3, nan: 0}
0


In [ ]:
#####'MasVnrType'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'MasVnrType')

val_count = unique_val_count(data, 'MasVnrType', unique_val)
print(val_count)

MasVnrType = data['MasVnrType'].replace(np.nan, 'None')
print(np.sum(MasVnrType.isnull()))

Missing value num: 24
Unique value:      ['BrkFace' 'None' 'Stone' 'BrkCmn' nan]
Value type:        object
The most frequent val is: None, it appeared 1742 times.
{'BrkFace': 879, 'None': 1742, 'Stone': 249, 'BrkCmn': 25, nan: 0}
0


In [ ]:
#####'BsmtQual'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'BsmtQual')
val_count = unique_val_count(data, 'BsmtQual', unique_val)
print(val_count)
BsmtQual = data['BsmtQual'].replace(np.nan, 'None')
print(np.sum(BsmtQual.isnull()))

Missing value num: 81
Unique value:      ['Gd' 'TA' 'Ex' nan 'Fa']
Value type:        object
The most frequent val is: TA, it appeared 1283 times.
{'Gd': 1209, 'TA': 1283, 'Ex': 258, nan: 0, 'Fa': 88}
0


In [ ]:
#####'BsmtCond'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'BsmtCond')
val_count = unique_val_count(data, 'BsmtCond', unique_val)
print(val_count)
BsmtCond = data['BsmtCond'].replace(np.nan, 'None')
print(np.sum(BsmtCond.isnull()))

Missing value num: 82
Unique value:      ['TA' 'Gd' nan 'Fa' 'Po']
Value type:        object
The most frequent val is: TA, it appeared 2606 times.
{'TA': 2606, 'Gd': 122, nan: 0, 'Fa': 104, 'Po': 5}
0


In [ ]:
#####'BsmtExposure'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'BsmtExposure')
val_count = unique_val_count(data, 'BsmtExposure', unique_val)
print(val_count)
BsmtExposure = data['BsmtExposure'].replace(np.nan, 'No')
print(np.sum(BsmtExposure.isnull()))

Missing value num: 82
Unique value:      ['No' 'Gd' 'Mn' 'Av' nan]
Value type:        object
The most frequent val is: No, it appeared 1904 times.
{'No': 1904, 'Gd': 276, 'Mn': 239, 'Av': 418, nan: 0}
0


In [ ]:
#####'BsmtFinType1'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'BsmtFinType1')
val_count = unique_val_count(data, 'BsmtFinType1', unique_val)
print(val_count)
BsmtFinType1 = data['BsmtFinType1'].replace(np.nan, 'None')
print(np.sum(BsmtFinType1.isnull()))

Missing value num: 79
Unique value:      ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
Value type:        object
The most frequent val is: Unf, it appeared 851 times.
{'GLQ': 849, 'ALQ': 429, 'Unf': 851, 'Rec': 288, 'BLQ': 269, nan: 0, 'LwQ': 154}
0


In [ ]:
#####'BsmtFinType2'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'BsmtFinType2')
val_count = unique_val_count(data, 'BsmtFinType2', unique_val)
print(val_count)
BsmtFinType2 = data['BsmtFinType2'].replace(np.nan, 'None')
print(np.sum(BsmtFinType2.isnull()))

Missing value num: 80
Unique value:      ['Unf' 'BLQ' nan 'ALQ' 'Rec' 'LwQ' 'GLQ']
Value type:        object
The most frequent val is: Unf, it appeared 2493 times.
{'Unf': 2493, 'BLQ': 68, nan: 0, 'ALQ': 52, 'Rec': 105, 'LwQ': 87, 'GLQ': 34}
0


In [ ]:
#####'Electrical'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'Electrical')
val_count = unique_val_count(data, 'Electrical', unique_val)
print(val_count)
Electrical = data['Electrical'].replace(np.nan, 'SBrKr')
print(np.sum(Electrical.isnull()))

Missing value num: 1
Unique value:      ['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix' nan]
Value type:        object
The most frequent val is: SBrkr, it appeared 2671 times.
{'SBrkr': 2671, 'FuseF': 50, 'FuseA': 188, 'FuseP': 8, 'Mix': 1, nan: 0}
0


In [ ]:
#####'KitchenQual'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'KitchenQual')
val_count = unique_val_count(data, 'KitchenQual', unique_val)
print(val_count)
KitchenQual = data['KitchenQual'].replace(np.nan, 'TA')
print(np.sum(KitchenQual.isnull()))

Missing value num: 1
Unique value:      ['Gd' 'TA' 'Ex' 'Fa' nan]
Value type:        object
The most frequent val is: TA, it appeared 1492 times.
{'Gd': 1151, 'TA': 1492, 'Ex': 205, 'Fa': 70, nan: 0}
0


In [ ]:
#####'Functional'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'Functional')
val_count = unique_val_count(data, 'Functional', unique_val)
print(val_count)
Functional = data['Functional'].replace(np.nan, 'Typ')
print(np.sum(Functional.isnull()))

Missing value num: 2
Unique value:      ['Typ' 'Min1' 'Maj1' 'Min2' 'Mod' 'Maj2' 'Sev' nan]
Value type:        object
The most frequent val is: Typ, it appeared 2717 times.
{'Typ': 2717, 'Min1': 65, 'Maj1': 19, 'Min2': 70, 'Mod': 35, 'Maj2': 9, 'Sev': 2, nan: 0}
0


In [ ]:
#####'FireplaceQu'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'FireplaceQu')
val_count = unique_val_count(data, 'FireplaceQu', unique_val)
print(val_count)
FireplaceQu = data['FireplaceQu'].replace(np.nan, 'None')
print(np.sum(FireplaceQu.isnull()))

Missing value num: 1420
Unique value:      [nan 'TA' 'Gd' 'Fa' 'Ex' 'Po']
Value type:        object
The most frequent val is: Gd, it appeared 744 times.
{nan: 0, 'TA': 592, 'Gd': 744, 'Fa': 74, 'Ex': 43, 'Po': 46}
0


In [ ]:
#####'GarageType'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'GarageType')
val_count = unique_val_count(data, 'GarageType', unique_val)
print(val_count)
GarageType = data['GarageType'].replace(np.nan, 'None')
print(np.sum(GarageType.isnull()))

Missing value num: 157
Unique value:      ['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
Value type:        object
The most frequent val is: Attchd, it appeared 1723 times.
{'Attchd': 1723, 'Detchd': 779, 'BuiltIn': 186, 'CarPort': 15, nan: 0, 'Basment': 36, '2Types': 23}
0


In [ ]:
#####'GarageFinish'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'GarageFinish')
val_count = unique_val_count(data, 'GarageFinish', unique_val)
print(val_count)
GarageFinish = data['GarageFinish'].replace(np.nan, 'None')
print(np.sum(GarageFinish.isnull()))

Missing value num: 159
Unique value:      ['RFn' 'Unf' 'Fin' nan]
Value type:        object
The most frequent val is: Unf, it appeared 1230 times.
{'RFn': 811, 'Unf': 1230, 'Fin': 719, nan: 0}
0


In [ ]:
#####'GarageQual'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'GarageQual')
val_count = unique_val_count(data, 'GarageQual', unique_val)
print(val_count)
GarageQual = data['GarageQual'].replace(np.nan, 'None')
print(np.sum(GarageQual.isnull()))

Missing value num: 159
Unique value:      ['TA' 'Fa' 'Gd' nan 'Ex' 'Po']
Value type:        object
The most frequent val is: TA, it appeared 2604 times.
{'TA': 2604, 'Fa': 124, 'Gd': 24, nan: 0, 'Ex': 3, 'Po': 5}
0


In [ ]:
#####'GarageCond'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'GarageCond')
val_count = unique_val_count(data, 'GarageCond', unique_val)
print(val_count)
GarageCond = data['GarageCond'].replace(np.nan, 'None')
print(np.sum(GarageCond.isnull()))

Missing value num: 159
Unique value:      ['TA' 'Fa' nan 'Gd' 'Po' 'Ex']
Value type:        object
The most frequent val is: TA, it appeared 2654 times.
{'TA': 2654, 'Fa': 74, nan: 0, 'Gd': 15, 'Po': 14, 'Ex': 3}
0


In [ ]:
#####'PoolQC'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'PoolQC')
val_count = unique_val_count(data, 'PoolQC', unique_val)
print(val_count)
PoolQC = data['PoolQC'].replace(np.nan, 'None')
print(np.sum(PoolQC.isnull()))

Missing value num: 2909
Unique value:      [nan 'Ex' 'Fa' 'Gd']
Value type:        object
The most frequent val is: Ex, it appeared 4 times.
{nan: 0, 'Ex': 4, 'Fa': 2, 'Gd': 4}
0


In [ ]:
#####'Fence'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'Fence')
val_count = unique_val_count(data, 'Fence', unique_val)
print(val_count)
Fence = data['Fence'].replace(np.nan, 'None')
print(np.sum(Fence.isnull()))

Missing value num: 2348
Unique value:      [nan 'MnPrv' 'GdWo' 'GdPrv' 'MnWw']
Value type:        object
The most frequent val is: MnPrv, it appeared 329 times.
{nan: 0, 'MnPrv': 329, 'GdWo': 112, 'GdPrv': 118, 'MnWw': 12}
0


In [ ]:
#####'MiscFeature'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'MiscFeature')
val_count = unique_val_count(data, 'MiscFeature', unique_val)
print(val_count)
MiscFeature = data['MiscFeature'].replace(np.nan, 'None')
print(np.sum(MiscFeature.isnull()))

Missing value num: 2814
Unique value:      [nan 'Shed' 'Gar2' 'Othr' 'TenC']
Value type:        object
The most frequent val is: Shed, it appeared 95 times.
{nan: 0, 'Shed': 95, 'Gar2': 5, 'Othr': 4, 'TenC': 1}
0


In [ ]:
#####'SaleType'#####
missing_val_num, unique_val, val_type = analysis_feature(data, 'SaleType')
val_count = unique_val_count(data, 'SaleType', unique_val)
print(val_count)
SaleType = data['SaleType'].replace(np.nan, 'WD')
print(np.sum(SaleType.isnull()))

Missing value num: 1
Unique value:      ['WD' 'New' 'COD' 'ConLD' 'ConLI' 'CWD' 'ConLw' 'Con' 'Oth' nan]
Value type:        object
The most frequent val is: WD, it appeared 2525 times.
{'WD': 2525, 'New': 239, 'COD': 87, 'ConLD': 26, 'ConLI': 9, 'CWD': 12, 'ConLw': 8, 'Con': 5, 'Oth': 7, nan: 0}
0


把以上已经填补了缺失值的特征重新组合成一张新表imputed_str_feature

In [ ]:
#      ['MSZoning', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual', 
#       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 
#       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType']

imputed_str_feature = pd.DataFrame()

imputed_str_feature['MSZoning'] = MSZoning
imputed_str_feature['Utilities'] = Utilities
imputed_str_feature['Exterior1st'] = Exterior1st
imputed_str_feature['Exterior2nd'] = Exterior2nd
imputed_str_feature['MasVnrType'] = MasVnrType
imputed_str_feature['BsmtQual'] = BsmtQual
imputed_str_feature['BsmtCond'] = BsmtCond
imputed_str_feature['BsmtExposure'] = BsmtExposure
imputed_str_feature['BsmtFinType1'] = BsmtFinType1
imputed_str_feature['BsmtFinType2'] = BsmtFinType2
imputed_str_feature['Electrical'] = Electrical
imputed_str_feature['KitchenQual'] = KitchenQual
imputed_str_feature['Functional'] = Functional
imputed_str_feature['FireplaceQu'] = FireplaceQu
imputed_str_feature['GarageType'] = GarageType
imputed_str_feature['GarageFinish'] = GarageFinish
imputed_str_feature['GarageQual'] = GarageQual
imputed_str_feature['GarageCond'] = GarageCond
imputed_str_feature['PoolQC'] = PoolQC
imputed_str_feature['Fence'] = Fence
imputed_str_feature['MiscFeature'] = MiscFeature
imputed_str_feature['SaleType'] = SaleType

print("The shape of imputed_str_feature: {}".format(imputed_str_feature.shape))
print(np.sum(imputed_str_feature.isnull()))
imputed_str_feature.head()

The shape of imputed_str_feature: (2919, 22)
MSZoning        0
Utilities       0
Exterior1st     0
Exterior2nd     0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Electrical      0
KitchenQual     0
Functional      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
SaleType        0
dtype: int64


,MSZoning,Utilities,Exterior1st,Exterior2nd,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature,SaleType
0,RL,AllPub,VinylSd,VinylSd,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Typ,None,Attchd,RFn,TA,TA,None,None,None,WD
1,RL,AllPub,MetalSd,MetalSd,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,None,None,None,WD
2,RL,AllPub,VinylSd,VinylSd,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,None,None,None,WD
3,RL,AllPub,Wd Sdng,Wd Shng,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,None,None,None,WD
4,RL,AllPub,VinylSd,VinylSd,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,None,None,None,WD


把没有缺失值的分类型特征和已填补了缺失值的分类型特征拼起来，组成没有缺失值的分类型特征集。

In [ ]:
# Combine string feature

no_missing_num_val_feature, no_missing_str_val_feature = sep_numerical_feature(data[no_missing_val_feature])
print("The shape of no_missing_str_val_feature: {}".format(data[no_missing_str_val_feature].shape))
print("The shape of imputed_str_feature: {}".format(imputed_str_feature.shape))
str_features = combine2df(data[no_missing_str_val_feature], imputed_str_feature)
print("The shape of str_features: {}".format(str_features.shape))
print(np.sum(str_features.isnull()))
str_features.head()

The shape of no_missing_str_val_feature: (2919, 20)
The shape of imputed_str_feature: (2919, 22)
The shape of str_features: (2919, 42)
Street           0
LotShape         0
LandContour      0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
ExterQual        0
ExterCond        0
Foundation       0
Heating          0
HeatingQC        0
CentralAir       0
PavedDrive       0
SaleCondition    0
MSZoning         0
Utilities        0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Electrical       0
KitchenQual      0
Functional       0
FireplaceQu      0
GarageType       0
GarageFinish     0
GarageQual       0
GarageCond       0
PoolQC           0
Fence            0
MiscFeature      0
SaleType         0
dtype: int64


,Street,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,ExterQual,ExterCond,Foundation,Heating,HeatingQC,CentralAir,PavedDrive,SaleCondition,MSZoning,Utilities,Exterior1st,Exterior2nd,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature,SaleType
0,Pave,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,Gd,TA,PConc,GasA,Ex,Y,Y,Normal,RL,AllPub,VinylSd,VinylSd,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Typ,None,Attchd,RFn,TA,TA,None,None,None,WD
1,Pave,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,TA,TA,CBlock,GasA,Ex,Y,Y,Normal,RL,AllPub,MetalSd,MetalSd,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,None,None,None,WD
2,Pave,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,Gd,TA,PConc,GasA,Ex,Y,Y,Normal,RL,AllPub,VinylSd,VinylSd,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,None,None,None,WD
3,Pave,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,TA,TA,BrkTil,GasA,Gd,Y,Y,Abnorml,RL,AllPub,Wd Sdng,Wd Shng,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,None,None,None,WD
4,Pave,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,Gd,TA,PConc,GasA,Ex,Y,Y,Normal,RL,AllPub,VinylSd,VinylSd,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,None,None,None,WD


In [ ]:
print(numerical_features.head())
print(numerical_features.shape)
print(str_features.head())
print(str_features.shape)

   Id  MSSubClass  LotArea  ...  GarageYrBlt  GarageCars  GarageArea
0   1          60     8450  ...       2003.0         2.0       548.0
1   2          20     9600  ...       1976.0         2.0       460.0
2   3          60    11250  ...       2001.0         2.0       608.0
3   4          70     9550  ...       1998.0         3.0       642.0
4   5          60    14260  ...       2000.0         3.0       836.0

[5 rows x 37 columns]
(2919, 37)
  Street LotShape LandContour LotConfig  ... PoolQC Fence MiscFeature SaleType
0   Pave      Reg         Lvl    Inside  ...   None  None        None       WD
1   Pave      Reg         Lvl       FR2  ...   None  None        None       WD
2   Pave      IR1         Lvl    Inside  ...   None  None        None       WD
3   Pave      IR1         Lvl    Corner  ...   None  None        None       WD
4   Pave      IR1         Lvl       FR2  ...   None  None        None       WD

[5 rows x 42 columns]
(2919, 42)


将数值型特征和分类特征重新拼成一张表

In [ ]:
imputed_train_data = combine2df(numerical_features, str_features)
print("The shape of imputed_train_data: {}".format(imputed_train_data.shape))
print(imputed_train_data.head())

The shape of imputed_train_data: (2919, 79)
   Id  MSSubClass  LotArea  OverallQual  ...  PoolQC  Fence  MiscFeature  SaleType
0   1          60     8450            7  ...    None   None         None        WD
1   2          20     9600            6  ...    None   None         None        WD
2   3          60    11250            7  ...    None   None         None        WD
3   4          70     9550            7  ...    None   None         None        WD
4   5          60    14260            8  ...    None   None         None        WD

[5 rows x 79 columns]


In [ ]:
imputed_data_columns = imputed_train_data.columns
print(imputed_data_columns)

Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'LotFrontage', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath',
       'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'Street',
       'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating',
       'HeatingQC', 'CentralAir', 'PavedDrive', 'SaleCondition', 'MSZoning',
       'Utilities', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'Bsm

对分类型特征进行one-hot编码

In [ ]:
imputed_onehot_train_data = pd.get_dummies(imputed_train_data)
print("The shape of imputed_onehot_train_data: {}".format(imputed_onehot_train_data.shape))
imputed_onehot_train_data.head()

The shape of imputed_onehot_train_data: (2919, 300)


,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea,Street_Grvl,Street_Pave,LotShape_IR1,...,GarageType_None,GarageFinish_Fin,GarageFinish_None,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,PoolQC_Ex,PoolQC_Fa,PoolQC_Gd,PoolQC_None,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_Gar2,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,1,60,8450,7,5,2003,2003,856,854,0,1710,2,1,3,1,8,0,0,61,0,0,0,0,0,2,2008,65.0,196.0,706.0,0.0,150.0,856.0,1.0,0.0,2003.0,2.0,548.0,0,1,0,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,2,20,9600,6,8,1976,1976,1262,0,0,1262,2,0,3,1,6,1,298,0,0,0,0,0,0,5,2007,80.0,0.0,978.0,0.0,284.0,1262.0,0.0,1.0,1976.0,2.0,460.0,0,1,0,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,3,60,11250,7,5,2001,2002,920,866,0,1786,2,1,3,1,6,1,0,42,0,0,0,0,0,9,2008,68.0,162.0,486.0,0.0,434.0,920.0,1.0,0.0,2001.0,2.0,608.0,0,1,1,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,4,70,9550,7,5,1915,1970,961,756,0,1717,1,0,3,1,7,1,0,35,272,0,0,0,0,2,2006,60.0,0.0,216.0,0.0,540.0,756.0,1.0,0.0,1998.0,3.0,642.0,0,1,1,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,5,60,14260,8,5,2000,2000,1145,1053,0,2198,2,1,4,1,9,1,192,84,0,0,0,0,0,12,2008,84.0,350.0,655.0,0.0,490.0,1145.0,1.0,0.0,2000.0,3.0,836.0,0,1,1,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
imputed_onehot_train_data.to_csv('clean_data.csv',index=False)